# A ideia desse notebook é prototipar uma solução para extrairmos a tabela requisitada

In [2]:
import requests
import json
import pandas as pd

## Pegando a lista de ids e parte dos dados

In [3]:
# request inicial
headers = {
	"Accept": "application/json",
	"Accept-Encoding": "gzip, deflate",
}
payload = {
"ano": "2021",
"comunicado": "1",
"situacao": "1",
"situacaoAndamento": True,
"situacaoEncerrada": False,
"tipoComunicado": "INICIAL",
"valorMobiliarioId": "15",
}
url = "http://web.cvm.gov.br/app/esforcosrestritos/consultarOfertaController/consultarValorMobiliario"
response = requests.post(url=url, 
                     json=payload, 
                     headers=headers)
dict_response = json.loads(response.content)

In [4]:
# parsing da request pra um df
df_inicial = pd.DataFrame.from_dict(dict_response)
df_inicial

,ofertasComunicadosTO,agrupador,totalVolume
0,"[{'ofertaTO': {'id': 17760, 'nomeResponsavel':...",Janeiro,"2.999.000,00"
1,"[{'ofertaTO': {'id': 18227, 'nomeResponsavel':...",Fevereiro,"0,00"
2,"[{'ofertaTO': {'id': 19550, 'nomeResponsavel':...",Março,"0,00"
3,"[{'ofertaTO': {'id': 20476, 'nomeResponsavel':...",Abril,"0,00"
4,"[{'ofertaTO': {'id': 20840, 'nomeResponsavel':...",Maio,"0,00"
5,"[{'ofertaTO': {'id': 21309, 'nomeResponsavel':...",Junho,"0,00"
6,"[{'ofertaTO': {'id': 21853, 'nomeResponsavel':...",Julho,"0,00"


In [5]:
# processa o df
list_of_dfs = []
for i in range(len(df_inicial)):
	list_of_dfs.append(pd.json_normalize(df_inicial['ofertasComunicadosTO'][i], sep='_'))
 
df_agregado = pd.concat(list_of_dfs, join='inner')
df_agregado.head(5)

,comunicadosTO,ofertaTO_id,ofertaTO_nomeResponsavel,ofertaTO_dataInicio,ofertaTO_numeroEmissao,ofertaTO_volumeOferta,ofertaTO_situacaoEncerrada,ofertaTO_situacaoAndamento,ofertaTO_tipoComunicado
0,"[{'id': 38399, 'descricao': 'Formulário Inicia...",17760,TRAVESSIA SECURITIZADORA DE CRÉDITOS FINANCEIR...,2021-01-11T00:00:00Z,1,"2.999.000,00",False,False,PARCIAL
1,"[{'id': 38397, 'descricao': 'Formulário Inicia...",17758,TRAVESSIA SECURITIZADORA DE CRÉDITOS FINANCEIR...,2021-01-11T00:00:00Z,1,"0,00",False,False,PARCIAL
2,"[{'id': 38398, 'descricao': 'Formulário Inicia...",17759,TRAVESSIA SECURITIZADORA DE CRÉDITOS FINANCEIR...,2021-01-11T00:00:00Z,1,"0,00",False,False,PARCIAL
3,"[{'id': 38521, 'descricao': 'Formulário Inicia...",17813,REVELO COMPANHIA SECURITIZADORA DE CREDITOS FI...,2021-01-18T00:00:00Z,1,"0,00",False,False,INICIAL
4,"[{'id': 38520, 'descricao': 'Formulário Inicia...",17812,REVELO COMPANHIA SECURITIZADORA DE CREDITOS FI...,2021-01-18T00:00:00Z,1,"0,00",False,False,INICIAL


## Agora precisamos de uma forma de fazer as requests a partir do id da tabela anterior, uma request por id (ou seja, uma por linha). Depois é só fazer um join

In [6]:
lista_ids = list(df_agregado['ofertaTO_id'])


In [7]:
list_of_dfs = []
for id_oferta in lista_ids:
    # tá lento (30s), podemos paralelizar essa request para acelerar
    url = "http://web.cvm.gov.br/app/esforcosrestritos/enviarFormularioParcial/getOfertaPorId/" + str(id_oferta)
    response = requests.get(url=url,
                             headers=headers)
    
    dict_response_2 = json.loads(response.content)
    
    list_of_dfs.append(pd.json_normalize(dict_response_2, sep='_'))    
     

In [8]:
df_agregado2 = pd.concat(list_of_dfs)

# Juntamos as duas colunas para obter a tabela que queríamos

In [9]:
df_agregado = df_agregado.reset_index(drop=True)
df_agregado2 = df_agregado2.reset_index(drop=True)

In [10]:
df_merged = df_agregado2.merge(df_agregado, right_on='ofertaTO_id', left_on="id", validate="one_to_one")

In [11]:
# ainda precisamos pegar o nome do lider via outra request
list_of_cnpjs = list(df_agregado2['lider_nrPfPj'])
list_of_responses = []
for cnpj in list_of_cnpjs:
    payload = {"cnpj": cnpj}
    url = "http://web.cvm.gov.br/app/esforcosrestritos/responsavel/getNomeResponsavelPorCnpj"
    response = requests.post(url=url,
                            json=payload,
                            headers=headers)
    dict_response = {"lider_nome": json.loads(response.content).strip(), "lider_nrPfPj": cnpj}
    list_of_responses.append(dict_response)

In [12]:
nomes_lideres = pd.DataFrame.from_dict(list_of_responses).drop_duplicates()

In [13]:
df_final = df_merged.merge(nomes_lideres, left_on="lider_nrPfPj", right_on="lider_nrPfPj", how='inner')

In [14]:
df_final